In [1]:
import pandas as pd
import numpy as np
import ast
import html
import string
import re
import nltk
from nltk.corpus import wordnet
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [2]:
scr_1 = pd.read_csv('./datasets/raw/20210405_2330_vaccine.csv')
scr_2 = pd.read_csv('./datasets/raw/20210406_2048_vaccine.csv')
scr_3 = pd.read_csv('./datasets/raw/20210505_2259_vaccine.csv')
scr_4 = pd.read_csv('./datasets/raw/20210507_0228_sinovac.csv')
scr_5 = pd.read_csv('./datasets/raw/20210507_0618_sinovac.csv')
scr_6 = pd.read_csv('./datasets/raw/20210507_1805_sinovac.csv')
scr_7 = pd.read_csv('./datasets/raw/20210507_2140_biontech.csv')

In [3]:
data = pd.concat([scr_1, scr_2, scr_3, scr_4, scr_5, scr_6, scr_7], axis = 0, ignore_index=True)
data.reset_index(inplace = True, drop = True)

In [4]:
data.tail()

,username,location,following,followers,totaltweets,whentweetcreated,whenacctcreated,retweets,text,hashtags
99995,JennellTebbett1,NaN,180,73,13959,2021-05-07 15:08:47,2015-10-18 22:56:29,1073,"BREAKING: Today, with @BioNTech_Group, we init...","[{'text': 'COVID19', 'indices': [129, 137]}]"
99996,orig80saddict,United States,4963,2247,31045,2021-05-07 15:08:44,2012-03-27 12:17:05,1,"You've been #vaccinated, and I've been #vaccin...","[{'text': 'vaccinated', 'indices': [12, 23]}, ..."
99997,aiyanna_19,NaN,167,145,476,2021-05-07 15:08:36,2019-11-26 21:52:45,1073,"BREAKING: Today, with @BioNTech_Group, we init...","[{'text': 'COVID19', 'indices': [129, 137]}]"
99998,DianeSzl,NaN,25,0,724,2021-05-07 15:08:33,2020-05-12 15:40:10,1073,"BREAKING: Today, with @BioNTech_Group, we init...","[{'text': 'COVID19', 'indices': [129, 137]}]"
99999,LibAthe80,Room 237,2556,2415,189951,2021-05-07 15:08:30,2011-02-07 20:57:57,10,"A new RNA vaccine is in the works, and it may ...",[]


In [5]:
data.count()

username            100000
location             68576
following           100000
followers           100000
totaltweets         100000
whentweetcreated    100000
whenacctcreated     100000
retweets            100000
text                100000
hashtags            100000
dtype: int64

In [6]:
# Drop duplicated rows:
data.drop_duplicates(keep=False, inplace=True)
# Drop empty tweets and missing values
data.dropna(subset=['text'], inplace=True)

In [7]:
data.count()

username            66253
location            44766
following           66253
followers           66253
totaltweets         66253
whentweetcreated    66253
whenacctcreated     66253
retweets            66253
text                66253
hashtags            66253
dtype: int64

In [8]:
data.reset_index(inplace = True, drop = True)

In [9]:
data.tail()

,username,location,following,followers,totaltweets,whentweetcreated,whenacctcreated,retweets,text,hashtags
66248,bitficus,Flatland,1526,438,3857,2021-05-07 15:08:50,2018-02-06 06:29:49,107,The Pfizer-BioNTech COVID-19 Vaccine has not b...,[]
66249,JennellTebbett1,NaN,180,73,13959,2021-05-07 15:08:47,2015-10-18 22:56:29,1073,"BREAKING: Today, with @BioNTech_Group, we init...","[{'text': 'COVID19', 'indices': [129, 137]}]"
66250,aiyanna_19,NaN,167,145,476,2021-05-07 15:08:36,2019-11-26 21:52:45,1073,"BREAKING: Today, with @BioNTech_Group, we init...","[{'text': 'COVID19', 'indices': [129, 137]}]"
66251,DianeSzl,NaN,25,0,724,2021-05-07 15:08:33,2020-05-12 15:40:10,1073,"BREAKING: Today, with @BioNTech_Group, we init...","[{'text': 'COVID19', 'indices': [129, 137]}]"
66252,LibAthe80,Room 237,2556,2415,189951,2021-05-07 15:08:30,2011-02-07 20:57:57,10,"A new RNA vaccine is in the works, and it may ...",[]


In [10]:
data = data.drop('totaltweets', axis=1)

In [11]:
def to_list(text):
    if pd.isnull(text):
        text = ''
    else:
        text = ast.literal_eval(text)
        # function to convert a string representation of a list to python list
    return text

In [12]:
data['hashtags'] = data['hashtags'].apply(lambda x: to_list(x))
# tek bir tweetteki her hashtagin dict olduğu bir liste döndürür

In [13]:
def extract_hashtags(hst_list):
    hashtags = []
    for i in range(0, len(hst_list)):
        hashtags.append(hst_list[i]['text'].lower())
    return hashtags

In [14]:
data['hashtags'] = data['hashtags'].apply(lambda x: extract_hashtags(x))

In [15]:
data.reset_index(inplace = True, drop = True)

In [16]:
data.tail()

,username,location,following,followers,whentweetcreated,whenacctcreated,retweets,text,hashtags
66248,bitficus,Flatland,1526,438,2021-05-07 15:08:50,2018-02-06 06:29:49,107,The Pfizer-BioNTech COVID-19 Vaccine has not b...,[]
66249,JennellTebbett1,NaN,180,73,2021-05-07 15:08:47,2015-10-18 22:56:29,1073,"BREAKING: Today, with @BioNTech_Group, we init...",[covid19]
66250,aiyanna_19,NaN,167,145,2021-05-07 15:08:36,2019-11-26 21:52:45,1073,"BREAKING: Today, with @BioNTech_Group, we init...",[covid19]
66251,DianeSzl,NaN,25,0,2021-05-07 15:08:33,2020-05-12 15:40:10,1073,"BREAKING: Today, with @BioNTech_Group, we init...",[covid19]
66252,LibAthe80,Room 237,2556,2415,2021-05-07 15:08:30,2011-02-07 20:57:57,10,"A new RNA vaccine is in the works, and it may ...",[]


In [17]:
scr_8 = pd.read_csv('./datasets/raw/biontech.csv')
scr_9 = pd.read_csv('./datasets/raw/sinovac.csv')
scr_10 = pd.read_csv('./datasets/raw/astrazeneca.csv')
scr_11 = pd.read_csv('./datasets/raw/moderna.csv')
scr_12 = pd.read_csv('./datasets/raw/sputnik.csv')
dummy = pd.concat([scr_8, scr_9, scr_10, scr_11, scr_12], axis=0, ignore_index=False)
dummy.reset_index(inplace = True, drop = True)

In [18]:
dummy.count()

Unnamed: 0        4287
id                3866
user_name         4287
user_location     4287
user_created      4287
user_followers    4287
user_friends      4287
date              4287
text              4287
hashtags          4287
retweets          4287
dtype: int64

In [19]:
dummy.tail()

,Unnamed: 0,id,user_name,user_location,user_created,user_followers,user_friends,date,text,hashtags,retweets
4282,69578,NaN,Carlos André Puras,"São Paulo, Brasil",2009-10-19 15:29:13,418,1051,2021-04-14 16:36:20,@sputnikvaccine #SputnikV #FechadoComBolsonar...,"['SputnikV', 'FechadoComBolsonaro', 'STF', 'Do...",0
4283,69616,NaN,Mustafa,india,2014-08-13 17:53:08,1474,5014,2021-04-14 15:08:55,Putin’s Once-Scorned Vaccine Now Favorite in P...,['SputnikV'],0
4284,69619,NaN,Sue,Lebanon,2011-09-17 19:22:58,392,614,2021-04-14 14:54:04,Today I got my first dose of the #SputnikV cor...,"['SputnikV', 'vaccines', 'CoronaVaccine', 'vac...",0
4285,69668,NaN,UNNATI - THE KNOWLEDGE HUB FOR CIVIL ASPIRANTS,"New Delhi, India",2017-04-09 09:31:10,1291,6,2021-04-14 12:01:03,Healing mankind! \n#SputnikV\n#Sputnikvaccine ...,"['SputnikV', 'Sputnikvaccine', 'sputnikvaccine...",45
4286,69705,NaN,Naga Sridhar,Hyderabad,2011-09-29 07:54:56,668,675,2021-04-14 08:47:39,#sputnikv #DRREDDYS #COVID # Sputinik V will b...,"['sputnikv', 'DRREDDYS', 'COVID']",0


In [20]:
dummy = dummy.drop('Unnamed: 0', axis=1)
dummy = dummy.drop('id', axis=1)

In [21]:
switch = {'user_name': 'username', 'user_location': 'location',
            'user_created': 'whenacctcreated', 'user_followers': 'followers',
            'user_friends': 'following', 'date': 'whentweetcreated',
        }
dummy = dummy.rename(columns=switch)

In [22]:
column_names = ['username', 'location', 'following', 'followers', 'whentweetcreated',
            'whenacctcreated', 'retweets', 'text', 'hashtags']
dummy = dummy.reindex(columns=column_names)

In [23]:
dummy.tail()

,username,location,following,followers,whentweetcreated,whenacctcreated,retweets,text,hashtags
4282,Carlos André Puras,"São Paulo, Brasil",1051,418,2021-04-14 16:36:20,2009-10-19 15:29:13,0,@sputnikvaccine #SputnikV #FechadoComBolsonar...,"['SputnikV', 'FechadoComBolsonaro', 'STF', 'Do..."
4283,Mustafa,india,5014,1474,2021-04-14 15:08:55,2014-08-13 17:53:08,0,Putin’s Once-Scorned Vaccine Now Favorite in P...,['SputnikV']
4284,Sue,Lebanon,614,392,2021-04-14 14:54:04,2011-09-17 19:22:58,0,Today I got my first dose of the #SputnikV cor...,"['SputnikV', 'vaccines', 'CoronaVaccine', 'vac..."
4285,UNNATI - THE KNOWLEDGE HUB FOR CIVIL ASPIRANTS,"New Delhi, India",6,1291,2021-04-14 12:01:03,2017-04-09 09:31:10,45,Healing mankind! \n#SputnikV\n#Sputnikvaccine ...,"['SputnikV', 'Sputnikvaccine', 'sputnikvaccine..."
4286,Naga Sridhar,Hyderabad,675,668,2021-04-14 08:47:39,2011-09-29 07:54:56,0,#sputnikv #DRREDDYS #COVID # Sputinik V will b...,"['sputnikv', 'DRREDDYS', 'COVID']"


In [24]:
data = pd.concat([data, dummy], axis=0, ignore_index=True)

In [25]:
data.count()

username            70540
location            49053
following           70540
followers           70540
whentweetcreated    70540
whenacctcreated     70540
retweets            70540
text                70540
hashtags            70540
dtype: int64

In [26]:
data.tail()

,username,location,following,followers,whentweetcreated,whenacctcreated,retweets,text,hashtags
70535,Carlos André Puras,"São Paulo, Brasil",1051,418,2021-04-14 16:36:20,2009-10-19 15:29:13,0,@sputnikvaccine #SputnikV #FechadoComBolsonar...,"['SputnikV', 'FechadoComBolsonaro', 'STF', 'Do..."
70536,Mustafa,india,5014,1474,2021-04-14 15:08:55,2014-08-13 17:53:08,0,Putin’s Once-Scorned Vaccine Now Favorite in P...,['SputnikV']
70537,Sue,Lebanon,614,392,2021-04-14 14:54:04,2011-09-17 19:22:58,0,Today I got my first dose of the #SputnikV cor...,"['SputnikV', 'vaccines', 'CoronaVaccine', 'vac..."
70538,UNNATI - THE KNOWLEDGE HUB FOR CIVIL ASPIRANTS,"New Delhi, India",6,1291,2021-04-14 12:01:03,2017-04-09 09:31:10,45,Healing mankind! \n#SputnikV\n#Sputnikvaccine ...,"['SputnikV', 'Sputnikvaccine', 'sputnikvaccine..."
70539,Naga Sridhar,Hyderabad,675,668,2021-04-14 08:47:39,2011-09-29 07:54:56,0,#sputnikv #DRREDDYS #COVID # Sputinik V will b...,"['sputnikv', 'DRREDDYS', 'COVID']"


In [27]:
# Collects mentioned users (@username)
def mentioned_users(string):
    usernames = re.findall('@[^\s]+', string)
    return usernames

In [28]:
# Create a new column and apply the function on the column 'text'
data['mentioned_users'] = data['text'].apply(lambda x: mentioned_users(x))
data.head()

,username,location,following,followers,whentweetcreated,whenacctcreated,retweets,text,hashtags,mentioned_users
0,LoloCraiger29,NaN,274,188,2021-04-05 20:28:21,2012-09-26 20:45:27,1,How many mom's out there have vaccination reco...,[],[]
1,Nexusswagon,Illinois,5002,1591,2021-04-05 20:28:20,2018-12-13 13:46:54,3185,"6+ months ago, I said that governments would p...",[],[]
2,mariemueller41,NaN,726,370,2021-04-05 20:28:20,2010-12-24 03:17:41,0,@jbirdbradyII My freshman year of college I wa...,[],"[@jbirdbradyII, @drdrew]"
3,BettyBunny23,Clotted cream first.,2866,2674,2021-04-05 20:28:20,2015-06-01 20:54:44,3,@Nigel_Farage I am slightly confused we we wer...,[],[@Nigel_Farage]
4,dwm2112,Galt's Gulch,350,163,2021-04-05 20:28:20,2009-04-28 19:44:48,5902,Requiring Americans to show proof of vaccinati...,[],[]


In [29]:
# Define the function to implement POS tagging:
def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [30]:
def cleaning(text):
    # remove html chars
    text = html.unescape(text)

    # remove urls
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', '', text)

    # remove usernames
    text = re.sub('@[^\s]+', '', text)

    # remove hashtags
    text = re.sub(r'#([^\s]+)', '', text)

    # remove &amp;
    text = text.replace('&amp;', 'and')

    # remove punctuations and lower text
    text = text.lower()
    text = [word.strip(string.punctuation) for word in text.split(" ")]

    # remove numbers
    text = [word for word in text if not any(c.isdigit() for c in word)]

    # remove stop words
    stop = stopwords.words('english')
    text = [x for x in text if x not in stop]

    # remove empty tokens
    text = [t for t in text if len(t) > 0]

    # pos tag text and lemmatize text
    pos_tags = pos_tag(text)
    text = [WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tags]

    # remove words with only one letter
    text = [t for t in text if len(t) > 1]

    # join all
    text = " ".join(text)
        
    return(text)

In [31]:
data['cleaned_text'] = data['text'].apply(lambda x: cleaning(x))

In [32]:
data.tail()

,username,location,following,followers,whentweetcreated,whenacctcreated,retweets,text,hashtags,mentioned_users,cleaned_text
70535,Carlos André Puras,"São Paulo, Brasil",1051,418,2021-04-14 16:36:20,2009-10-19 15:29:13,0,@sputnikvaccine #SputnikV #FechadoComBolsonar...,"['SputnikV', 'FechadoComBolsonaro', 'STF', 'Do...",[@sputnikvaccine],brazilian supreme court communists try…
70536,Mustafa,india,5014,1474,2021-04-14 15:08:55,2014-08-13 17:53:08,0,Putin’s Once-Scorned Vaccine Now Favorite in P...,['SputnikV'],"[@drreddys, @sputnikvaccine]",putin’s once-scorned vaccine favorite pandemic...
70537,Sue,Lebanon,614,392,2021-04-14 14:54:04,2011-09-17 19:22:58,0,Today I got my first dose of the #SputnikV cor...,"['SputnikV', 'vaccines', 'CoronaVaccine', 'vac...",[@sputnikvaccine],today get first dose coronavirus vaccine.\n
70538,UNNATI - THE KNOWLEDGE HUB FOR CIVIL ASPIRANTS,"New Delhi, India",6,1291,2021-04-14 12:01:03,2017-04-09 09:31:10,45,Healing mankind! \n#SputnikV\n#Sputnikvaccine ...,"['SputnikV', 'Sputnikvaccine', 'sputnikvaccine...",[],heal mankind \n\n
70539,Naga Sridhar,Hyderabad,675,668,2021-04-14 08:47:39,2011-09-29 07:54:56,0,#sputnikv #DRREDDYS #COVID # Sputinik V will b...,"['sputnikv', 'DRREDDYS', 'COVID']",[],sputinik available india current quarter dr re...


In [33]:
import preprocessor as p

def preprocess_tweet(row):
    p.set_options(p.OPT.EMOJI, p.OPT.SMILEY, p.OPT.RESERVED)
    text = row['cleaned_text']
    text = p.clean(text)
    return text

In [34]:
#apply the function to text column, axis = 1 for all rows
data['cleaned_text'] = data.apply(preprocess_tweet, axis=1)

In [35]:
data.head(10)['cleaned_text']

0    many mom's vaccination record child big deal v...
1    month ago say government would push vaccine pa...
2    freshman year college wasnt allow enroll class...
3    slightly confuse tell vaccine end life back no...
4    require american show proof vaccination vaccin...
5    tomorrow morning a.m state health official hol...
6    maryland expand vaccine eligibility adult next...
7    fed lead poll majority convince anyone robyn f...
8    public health england advise pregnant woman pl...
9                   sign help find vaccine appointment
Name: cleaned_text, dtype: object

In [36]:
data.tail(10)['cleaned_text']

70530               brazilian supreme court communists try
70531               brazilian supreme court communists try
70532               brazilian supreme court communists try
70533               brazilian supreme court communists try
70534               brazilian supreme court communists try
70535               brazilian supreme court communists try
70536    putins once-scorned vaccine favorite pandemic ...
70537            today get first dose coronavirus vaccine.
70538                                         heal mankind
70539    sputinik available india current quarter dr re...
Name: cleaned_text, dtype: object

In [37]:
# Drop duplicated rows:
data.drop_duplicates(subset='cleaned_text', keep="first", inplace=True)

In [38]:
pd.set_option('display.max_colwidth', None)

In [39]:
data.tail(10)['cleaned_text']

70508                    need questionable foremost political tool div
70509                           sec finally approve sputnik vaccine!!!
70510                                          please help brazil need
70511                      brazilian supreme court communists try dest
70520                         brazilian supreme court communists tryin
70529                           brazilian supreme court communists try
70536    putins once-scorned vaccine favorite pandemic fight bloomberg
70537                        today get first dose coronavirus vaccine.
70538                                                     heal mankind
70539              sputinik available india current quarter dr reddy's
Name: cleaned_text, dtype: object

In [40]:
data.count()

username            24083
location            18074
following           24083
followers           24083
whentweetcreated    24083
whenacctcreated     24083
retweets            24083
text                24083
hashtags            24083
mentioned_users     24083
cleaned_text        24083
dtype: int64

In [41]:
# Check out the shape again and reset_index
print(data.shape)
data.reset_index(inplace = True, drop = True)

# Check out data.tail() to validate index has been reset
data.tail()

(24083, 11)


,username,location,following,followers,whentweetcreated,whenacctcreated,retweets,text,hashtags,mentioned_users,cleaned_text
24078,Carlos André Puras,"São Paulo, Brasil",1051,418,2021-04-14 16:37:16,2009-10-19 15:29:13,1,@sputnikvaccine #SputnikV #FechadoComBolsonaro #STF #DoriaPatife Brazilian Supreme Court and Communists are try… https://t.co/SmmjcHAXBh,"['SputnikV', 'FechadoComBolsonaro', 'STF', 'DoriaPatife']",[@sputnikvaccine],brazilian supreme court communists try
24079,Mustafa,india,5014,1474,2021-04-14 15:08:55,2014-08-13 17:53:08,0,Putin’s Once-Scorned Vaccine Now Favorite in Pandemic Fight - Bloomberg @drreddys @sputnikvaccine #SputnikV… https://t.co/EMyVMjJ9Jj,['SputnikV'],"[@drreddys, @sputnikvaccine]",putins once-scorned vaccine favorite pandemic fight bloomberg
24080,Sue,Lebanon,614,392,2021-04-14 14:54:04,2011-09-17 19:22:58,0,Today I got my first dose of the #SputnikV coronavirus vaccine.\n@sputnikvaccine #vaccines #CoronaVaccine #vaccine… https://t.co/LHtjGBGpyL,"['SputnikV', 'vaccines', 'CoronaVaccine', 'vaccine']",[@sputnikvaccine],today get first dose coronavirus vaccine.
24081,UNNATI - THE KNOWLEDGE HUB FOR CIVIL ASPIRANTS,"New Delhi, India",6,1291,2021-04-14 12:01:03,2017-04-09 09:31:10,45,Healing mankind! \n#SputnikV\n#Sputnikvaccine \n#sputnikvaccinelifesaver \n#COVID19 \n#CoronavirusIndia \n#COVIDUpdates https://t.co/DXnwCls8G4,"['SputnikV', 'Sputnikvaccine', 'sputnikvaccinelifesaver', 'COVID19', 'CoronavirusIndia', 'COVIDUpdates']",[],heal mankind
24082,Naga Sridhar,Hyderabad,675,668,2021-04-14 08:47:39,2011-09-29 07:54:56,0,#sputnikv #DRREDDYS #COVID # Sputinik V will be available in India in the current quarter: Dr Reddy's,"['sputnikv', 'DRREDDYS', 'COVID']",[],sputinik available india current quarter dr reddy's


# LOCATIONS

In [42]:
def remove_nan(text):
    if pd.isnull(text):
        text = 'n.a.'
    else:
        text = text.lower()
    return text

In [43]:
data['location'] = data['location'].apply(lambda x:remove_nan(x))

In [44]:
data['location'].value_counts()

n.a.                  6009
united states          324
los angeles, ca        204
new york, ny           203
canada                 174
                      ... 
nairobi.kenya            1
atx/nyc                  1
essex girl in kent       1
south east asia          1
in transit               1
Name: location, Length: 7976, dtype: int64

In [45]:
sid = SentimentIntensityAnalyzer()

In [46]:
data['sent_by_lexicon'] = data['cleaned_text'].apply(lambda x: sid.polarity_scores(x))

# Keep only the compound scores under the column 'Sentiment'
data = pd.concat([data.drop(['sent_by_lexicon'], axis = 1), data['sent_by_lexicon'].apply(pd.Series)], axis = 1)

In [47]:
data.head()

,username,location,following,followers,whentweetcreated,whenacctcreated,retweets,text,hashtags,mentioned_users,cleaned_text,neg,neu,pos,compound
0,LoloCraiger29,n.a.,274,188,2021-04-05 20:28:21,2012-09-26 20:45:27,1,How many mom's out there have vaccination records for your children? What is the big deal with vaccination passports? They show you have had the vaccine and also you have resect for people with underlining conditions. I guess that's why Republicans are carrying on!!!,[],[],many mom's vaccination record child big deal vaccination passports show vaccine also resect people underlining condition guess that's republican carry,0.000,1.000,0.000,0.0000
1,Nexusswagon,illinois,5002,1591,2021-04-05 20:28:20,2018-12-13 13:46:54,3185,"6+ months ago, I said that governments would push for 'vaccine passports' or 'freedom passes' as part of 'The New Normal/Great Reset'.\n\nI was told that I was a 'crazy conspiracy theorist'.\n\nNow, those same people are trying to convince me that it's a good idea. \n\nMuppets.",[],[],month ago say government would push vaccine passport freedom pass part new normal/great reset'. i tell crazy conspiracy theorist'. now people try convince good idea muppets,0.166,0.573,0.261,0.5106
2,mariemueller41,n.a.,726,370,2021-04-05 20:28:20,2010-12-24 03:17:41,0,@jbirdbradyII My freshman year of college I wasn’t allowed to enroll in classes because my 2nd MMR vaccine was given when I was 360 days old and not 1 year. So at 18 I got a baby vaccine. Also @drdrew is an intentional idiot. He just needed some attention today. Here it is sweetie.,[],"[@jbirdbradyII, @drdrew]",freshman year college wasnt allow enroll class mmr vaccine give day old year get baby vaccine also intentional idiot need attention today sweetie,0.176,0.710,0.114,-0.1940
3,BettyBunny23,clotted cream first.,2866,2674,2021-04-05 20:28:20,2015-06-01 20:54:44,3,@Nigel_Farage I am slightly confused we we were told the vaccine was the be all and end all !! Life back to normal so why are they bringing in the vaccine passport when 90% of the country would’ve been vaccinated ? It sounds like they were not quite as confident as they were .,[],[@Nigel_Farage],slightly confuse tell vaccine end life back normal bringing vaccine passport country wouldve vaccinated sound like quite confident,0.071,0.664,0.265,0.6582
4,dwm2112,galt's gulch,350,163,2021-04-05 20:28:20,2009-04-28 19:44:48,5902,"Requiring Americans to show proof of vaccination, or a ‘vaccine passport,’ in order to engage in society is outrageous and un-American. This isn’t about containing a virus, but is just another dangerous idea from the Left in their quest for more power and control.",[],[],"require american show proof vaccination vaccine passport, order engage society outrageous un-american isnt contain virus another dangerous idea leave quest power control",0.264,0.650,0.087,-0.5994


In [48]:
# Create a new column called 'sentiment_class'
sentimentclass_list = []

for i in range(0, len(data)):
    
    # current 'compound' score:
    curr_compound = data.iloc[i,:]['compound']
    
    if (curr_compound <= 1.0 and curr_compound >= 0.10):
        sentimentclass_list.append(1)
    elif (curr_compound < 0.10 and curr_compound > -0.10):
        sentimentclass_list.append(0)
    elif (curr_compound <= -0.10 and curr_compound >= -1.00):
        sentimentclass_list.append(-1)

# Add the new column 'sentiment_class' to the dataframe
data['sentiment'] = sentimentclass_list

In [49]:
data.head()

,username,location,following,followers,whentweetcreated,whenacctcreated,retweets,text,hashtags,mentioned_users,cleaned_text,neg,neu,pos,compound,sentiment
0,LoloCraiger29,n.a.,274,188,2021-04-05 20:28:21,2012-09-26 20:45:27,1,How many mom's out there have vaccination records for your children? What is the big deal with vaccination passports? They show you have had the vaccine and also you have resect for people with underlining conditions. I guess that's why Republicans are carrying on!!!,[],[],many mom's vaccination record child big deal vaccination passports show vaccine also resect people underlining condition guess that's republican carry,0.000,1.000,0.000,0.0000,0
1,Nexusswagon,illinois,5002,1591,2021-04-05 20:28:20,2018-12-13 13:46:54,3185,"6+ months ago, I said that governments would push for 'vaccine passports' or 'freedom passes' as part of 'The New Normal/Great Reset'.\n\nI was told that I was a 'crazy conspiracy theorist'.\n\nNow, those same people are trying to convince me that it's a good idea. \n\nMuppets.",[],[],month ago say government would push vaccine passport freedom pass part new normal/great reset'. i tell crazy conspiracy theorist'. now people try convince good idea muppets,0.166,0.573,0.261,0.5106,1
2,mariemueller41,n.a.,726,370,2021-04-05 20:28:20,2010-12-24 03:17:41,0,@jbirdbradyII My freshman year of college I wasn’t allowed to enroll in classes because my 2nd MMR vaccine was given when I was 360 days old and not 1 year. So at 18 I got a baby vaccine. Also @drdrew is an intentional idiot. He just needed some attention today. Here it is sweetie.,[],"[@jbirdbradyII, @drdrew]",freshman year college wasnt allow enroll class mmr vaccine give day old year get baby vaccine also intentional idiot need attention today sweetie,0.176,0.710,0.114,-0.1940,-1
3,BettyBunny23,clotted cream first.,2866,2674,2021-04-05 20:28:20,2015-06-01 20:54:44,3,@Nigel_Farage I am slightly confused we we were told the vaccine was the be all and end all !! Life back to normal so why are they bringing in the vaccine passport when 90% of the country would’ve been vaccinated ? It sounds like they were not quite as confident as they were .,[],[@Nigel_Farage],slightly confuse tell vaccine end life back normal bringing vaccine passport country wouldve vaccinated sound like quite confident,0.071,0.664,0.265,0.6582,1
4,dwm2112,galt's gulch,350,163,2021-04-05 20:28:20,2009-04-28 19:44:48,5902,"Requiring Americans to show proof of vaccination, or a ‘vaccine passport,’ in order to engage in society is outrageous and un-American. This isn’t about containing a virus, but is just another dangerous idea from the Left in their quest for more power and control.",[],[],"require american show proof vaccination vaccine passport, order engage society outrageous un-american isnt contain virus another dangerous idea leave quest power control",0.264,0.650,0.087,-0.5994,-1


In [50]:
data = data.drop('neg', axis=1)
data = data.drop('neu', axis=1)
data = data.drop('pos', axis=1)
data = data.drop('compound', axis=1)

In [56]:
data.head()

,username,location,following,followers,whentweetcreated,whenacctcreated,retweets,text,hashtags,mentioned_users,cleaned_text,sentiment
0,LoloCraiger29,n.a.,274,188,2021-04-05 20:28:21,2012-09-26 20:45:27,1,How many mom's out there have vaccination records for your children? What is the big deal with vaccination passports? They show you have had the vaccine and also you have resect for people with underlining conditions. I guess that's why Republicans are carrying on!!!,[],[],many mom's vaccination record child big deal vaccination passports show vaccine also resect people underlining condition guess that's republican carry,Neutral
1,Nexusswagon,illinois,5002,1591,2021-04-05 20:28:20,2018-12-13 13:46:54,3185,"6+ months ago, I said that governments would push for 'vaccine passports' or 'freedom passes' as part of 'The New Normal/Great Reset'.\n\nI was told that I was a 'crazy conspiracy theorist'.\n\nNow, those same people are trying to convince me that it's a good idea. \n\nMuppets.",[],[],month ago say government would push vaccine passport freedom pass part new normal/great reset'. i tell crazy conspiracy theorist'. now people try convince good idea muppets,Positive
2,mariemueller41,n.a.,726,370,2021-04-05 20:28:20,2010-12-24 03:17:41,0,@jbirdbradyII My freshman year of college I wasn’t allowed to enroll in classes because my 2nd MMR vaccine was given when I was 360 days old and not 1 year. So at 18 I got a baby vaccine. Also @drdrew is an intentional idiot. He just needed some attention today. Here it is sweetie.,[],"[@jbirdbradyII, @drdrew]",freshman year college wasnt allow enroll class mmr vaccine give day old year get baby vaccine also intentional idiot need attention today sweetie,Negative
3,BettyBunny23,clotted cream first.,2866,2674,2021-04-05 20:28:20,2015-06-01 20:54:44,3,@Nigel_Farage I am slightly confused we we were told the vaccine was the be all and end all !! Life back to normal so why are they bringing in the vaccine passport when 90% of the country would’ve been vaccinated ? It sounds like they were not quite as confident as they were .,[],[@Nigel_Farage],slightly confuse tell vaccine end life back normal bringing vaccine passport country wouldve vaccinated sound like quite confident,Positive
4,dwm2112,galt's gulch,350,163,2021-04-05 20:28:20,2009-04-28 19:44:48,5902,"Requiring Americans to show proof of vaccination, or a ‘vaccine passport,’ in order to engage in society is outrageous and un-American. This isn’t about containing a virus, but is just another dangerous idea from the Left in their quest for more power and control.",[],[],"require american show proof vaccination vaccine passport, order engage society outrageous un-american isnt contain virus another dangerous idea leave quest power control",Negative


In [55]:
data['sentiment'].replace(to_replace = 1, value = 'Positive', inplace = True)
data['sentiment'].replace(to_replace = -1, value = 'Negative', inplace = True)
data['sentiment'].replace(to_replace = 0, value = 'Neutral', inplace = True)

In [57]:
data.to_csv('./datasets/cleaned/vaccines-labeled-lexicon.csv', index = False)